In [1]:
import os
import time
import sys
import h5py
import re
import tensorflow  as tf
import numpy       as np
import pandas      as pd
from matplotlib    import pyplot as plt 
from os            import path
from tqdm          import tqdm

print("="*50) 
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [2]:
WORKSPACE_PATH      = os.environ['WORKSPACE_PATH']
SurQCTFldr          = WORKSPACE_PATH + '/SurQCT/surqct/'

RatesType           = 'KExcit'
ExcitType           = 'KInel'
NNRunIdx            = 21
# ExcitType           = 'KExch'
# NNRunIdx            = 1

System              = 'N3_NASA'  
ZeroDRunIdx         = NNRunIdx

# PathToRunFld        = SurQCTFldr + '/../' + RatesType + '/all_temperatures_nondim/' + ExcitType + '/' 
PathToRunFld        = SurQCTFldr + '/../' + RatesType + '_N3_TransLearn/all_temperatures_nondim/' + ExcitType + '/' 

TTranVec            = [5000.0]
# Molecules           = ['O2','O2']
# Atoms               = ['O','O']
Molecules           = ['N2','N2']
Atoms               = ['N','N']

MinValueTrain = 1.e-15

In [3]:
print("\n[SurQCT]: Loading Modules and Functions ...")

sys.path.insert(0, SurQCTFldr  + '/src/Reading/')
from Reading  import read_levelsdata, read_diatdata

InputFld = PathToRunFld + '/Run_' + str(NNRunIdx) + '/'
print(InputFld)
sys.path.insert(0, InputFld)


[SurQCT]: Loading Modules and Functions ...
/home/venturi/WORKSPACE//SurQCT/surqct//../KExcit_N3_TransLearn/all_temperatures_nondim/KInel//Run_21/


In [4]:
print("\n[SurQCT]: Keep Loading Modules and Functions...")
from SurQCT_Input import inputdata

print("\n[SurQCT]: Initializing Input ...")
InputData    = inputdata(WORKSPACE_PATH, SurQCTFldr)

Prefix                    = 'Run_'
InputData.NNRunIdx        = NNRunIdx
InputData.PathToRunFld    = InputData.PathToRunFld+'/'+Prefix+str(InputData.NNRunIdx)
InputData.PathToFigFld    = InputData.PathToRunFld+'/'+InputData.PathToFigFld
InputData.PathToParamsFld = InputData.PathToRunFld+'/'+InputData.PathToParamsFld
InputData.PathToDataFld   = InputData.PathToRunFld+'/Data/'                                                               
InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/' 

## N2
# InputData.PathToHDF5File  = InputData.WORKSPACE_PATH  + '/Air_Database/HDF5_Database/N3_NASA.hdf5'
# InputData.Molecules       = ['N2','N2'] 
# InputData.PathToLevelsFile= [InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_GroundState_log_nd.csv',
#                              InputData.WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/N2_LeRoy_GroundState_log_nd.csv']
# InputData.PathToDiatFile  = [InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp',
#                             InputData.WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp']   

InputData.KineticFldr     = InputData.WORKSPACE_PATH+'/Air_Database/Run_0D_surQCT/database/kinetics/transfer_'+System+'_Active_Run'+str(ZeroDRunIdx)+'/'
InputData.QCTKineticFldr  = WORKSPACE_PATH+'/Air_Database/Run_0D_semi/database/kinetics/'+System+'/'


[SurQCT]: Keep Loading Modules and Functions...

[SurQCT]: Initializing Input ...


In [5]:
print("\n[SurQCT]: Loading Final Modules ... ")

sys.path.insert(0, SurQCTFldr  + '/src/Model/' + InputData.ApproxModel + '/')
from Model import model


InputData.DefineModelIntFlg  = 1
InputData.TrainIntFlg        = 0
NN_KExcit                    = model(InputData, InputData.PathToRunFld, None, None)
NN_KExcit.load_params(InputData.PathToParamsFld)


[SurQCT]: Loading Final Modules ... 
[SurQCT]:   Variables Selected for Training:
[SurQCT]:     x =  ['log_EVib_i', 'log_ERot_i', 'ri_i', 'log_rorMin_i', 'TTran_i', 'log_EVib_Delta', 'log_ERot_Delta', 'ri_Delta', 'log_rorMin_Delta', 'TTran_Delta']
[SurQCT]:     y =  KExcit
[SurQCT]:   Defining ML Model from Scratch
[SurQCT]:   Compiling ML Model with Loss and Optimizer
[SurQCT]:   Saving ML Model in File: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit_N3_TransLearn/all_temperatures_nondim/KInel//Run_21/MyModel
INFO:tensorflow:Assets written to: /home/venturi/WORKSPACE//SurQCT/surqct//../KExcit_N3_TransLearn/all_temperatures_nondim/KInel//Run_21/MyModel/assets
[SurQCT]:   Summarizing ML Model Structure:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                 

In [6]:
OtherVar           = InputData.OtherVar
xVarsVec_i         = InputData.xVarsVec_i + ['vqn','jqn']
xVarsVec_Delta     = InputData.xVarsVec_Delta
xVarsVec           = list(set(xVarsVec_i) | set(xVarsVec_Delta))
print('[SurQCT]:   Reading Variables: ', xVarsVec)

InputData.MultFact = 1.e+9
InputData.MinValueTrain      = 1.e-16 * InputData.MultFact
InputData.MinValueTest       = 1.e-16 * InputData.MultFact
NoiseSD            = 1.e-13 * InputData.MultFact

NMolecules         = len(InputData.PathToLevelsFile)

[SurQCT]:   Reading Variables:  ['vqn', 'log_rorMin', 'log_EVib', 'jqn', 'ri', 'log_ERot']


In [7]:
LevelsData = []
DiatData   = []
NLevels    = []
for iMol in range(NMolecules):

    LevelsDataTemp = read_levelsdata(InputData.PathToLevelsFile[iMol], xVarsVec, '')
    LevelsData.append(LevelsDataTemp)

    DiatDataTemp = read_diatdata(InputData.PathToDiatFile[iMol], InputData.Molecules[iMol], np.array(TTranVec), np.array(TTranVec))
    DiatData.append(DiatDataTemp)

    NLevelsTemp    = LevelsDataTemp.shape[0]
    NLevels.append(NLevelsTemp)

[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//Air_Database/Run_0D/database/levels/N2_LeRoy_log_nd.csv
[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp


In [8]:
#========================================================================================================
def read_kexcitdata_PLATO(InputData, PathToKineticFile, TTra):
    print(PathToKineticFile)
    Rates = np.zeros((NLevels[0],NLevels[0]))
    i2j_dict = {}
    with open(PathToKineticFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('N2|N|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            jIdx    = int(linestemp[8]);
            RateTemp = float(linestemp[12]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates, i2j_dict

#========================================================================================================
def write_predictiondata(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):

    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,+0.0000E+00,+0.0000E+00,6\n' % KVec[i]
                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics



# Reading QCT Kinetics used in Plato

In [9]:
if(ExcitType == 'KInel'): 
    PathToKineticFile  = InputData.QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Inel.dat'
elif(ExcitType == 'KExch'): 
    PathToKineticFile  = InputData.QCTKineticFldr + '/T' + str(int(TTranVec[0])) + 'K/Exch_Type1.dat'
    
QCTRates, i2j_dict = read_kexcitdata_PLATO(InputData,PathToKineticFile,TTranVec[0])

/home/venturi/WORKSPACE//Air_Database/Run_0D_semi/database/kinetics/N3_NASA//T5000K/Inel.dat


# Writing QCT Transitions only 

In [10]:
for TTran in TTranVec:
#     QCTRates, i2j_dict = read_kexcitdata_PLATO(InputData,InputData.QCTKineticFldr,TTran)
    
    ### Opening Files                                                                                                                                                                 
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):
        KineticFile_KInel = PathToKineticFldr + '/Inel_QCTTransitions_all.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_QCTTransitions_all.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)
            
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(i2j_dict.keys(), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.02)
        
        jIdxVec  = i2j_dict[iIdx]
        jNLevels = len(jIdxVec)
    
        ### FWD Rates
        iiIdxVec              = [iIdx]*jNLevels
    
        TTranVec              = np.ones((jNLevels))*TTran
        TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
        TTranDataTemp.index   = jIdxVec

    
        iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
        iLevelsDataTemp.index = jIdxVec

        jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        if (OtherVar == '_Delta'):
            jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
        else:
            jLevelsDataTemp   = jLevelsDataTemp
        jLevelsDataTemp.index = jIdxVec
    
        kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
        kLevelsDataTemp.index = jIdxVec

    
        iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    
        jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
        jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

        kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
        kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
    
        xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
    
   
        if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
        
            if(ExcitType == 'KInel'):   
                KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
            
            elif(ExcitType == 'KExch'):      
                KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)
                           
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)
    

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/9397 [00:00<?, ?it/s]

[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/transfer_N3_NASA_Active_Run21//T5000K/Inel_QCTTransitions_all.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|██████████| 9397/9397 [11:16<00:00, 13.89it/s]

[SurQCT]:   Closing Kinetics File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/transfer_N3_NASA_Active_Run21//T5000K/Inel_QCTTransitions_all.dat


# Replacing v=0 and v=1 Rates with QCT Rates

In [11]:
vqnVec = DiatData[0]['vqn'].to_numpy(int)
jqnVec = DiatData[0]['jqn'].to_numpy(int)

In [12]:
try:
    os.makedirs(InputData.KineticFldr)
except OSError as e:
    pass

iLevelsList = [340]

for TTran in TTranVec:
    print('[SurQCT]:    TTran = ', TTran)

    ### Opening Files
    PathToKineticFldr = InputData.KineticFldr + '/T' + str(int(TTran)) + 'K'
    try:
        os.makedirs(PathToKineticFldr)
    except OSError as e:
        pass

    if(ExcitType == 'KInel'):   
        KineticFile_KInel = PathToKineticFldr + '/Inel_QCTRate_v0v1_all.dat'
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, None, -1, Molecules, Atoms, None, None, None)
    elif(ExcitType == 'KExch'):   
        KineticFile_KExch = PathToKineticFldr + '/Exch_Type1_QCTRate_v0v1_all.dat'
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, None, -1, Molecules, Atoms, None, None, None)

    ### Loop on Initial States
    Str = 'q_'+str(int(TTran))
    for iIdx in tqdm(range(NLevels[0]), desc='[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes'):
        time.sleep(0.001)
        
        if (InputData.ExoEndoFlg):
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] < DiatData[0]['EInt'].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1]['EInt'].to_numpy()[jIdx] >= DiatData[0]['EInt'].to_numpy()[iIdx])]
        else:
            jIdxVec           = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    > DiatData[0][Str].to_numpy()[iIdx])]
            jIdxVecNo         = [jIdx for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
            RatioNo           = [DiatData[0][Str].to_numpy()[iIdx]/DiatData[1][Str].to_numpy()[jIdx] for jIdx in np.arange(NLevels[1]) if (DiatData[1][Str].to_numpy()[jIdx]    <= DiatData[0][Str].to_numpy()[iIdx])]
        jNLevels              = len(jIdxVec)
        jNLevelsNo            = len(jIdxVecNo)

        # Writing QCT Rates
        if(vqnVec[iIdx]==0 or vqnVec[iIdx]==1):
            if(ExcitType == 'KInel'):
                jIdxVecQCT = i2j_dict[iIdx+1]
                KInel_NN_FWD = QCTRates[iIdx+1,jIdxVecQCT]
                csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx+1, jIdxVecQCT, KInel_NN_FWD)
            elif(ExcitType == 'KExch'):
                jIdxVecQCT = i2j_dict[iIdx+2]    
                KExch_NN_FWD = QCTRates[iIdx+2,jIdxVecQCT]
                csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx+2, jIdxVecQCT, KExch_NN_FWD)

        # Writing NN Rates    
        else:
            iiIdxVec              = [iIdx]*jNLevels
    
            TTranVec              = np.ones((jNLevels))*TTran
            TTranDataTemp         = pd.DataFrame({'TTran': TTranVec})
            TTranDataTemp.index   = jIdxVec

    
            iLevelsDataTemp       = LevelsData[0].iloc[iiIdxVec,:].copy()
            iLevelsDataTemp.index = jIdxVec

            jLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            if (OtherVar == '_Delta'):
                jLevelsDataTemp   = iLevelsDataTemp.subtract(jLevelsDataTemp) 
            else:
                jLevelsDataTemp   = jLevelsDataTemp
            jLevelsDataTemp.index = jIdxVec
    
            kLevelsDataTemp       = LevelsData[1].iloc[jIdxVec,:].copy()        
            kLevelsDataTemp.index = jIdxVec

    
            iLevelsData           = pd.concat([iLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            iLevelsData.columns   = [(VarName + '_i') for VarName in iLevelsData.columns]
    
            jLevelsData           = pd.concat([jLevelsDataTemp[xVarsVec_Delta], TTranDataTemp], axis=1)
            jLevelsData.columns   = [(VarName + OtherVar) for VarName in jLevelsData.columns]

            kLevelsData           = pd.concat([kLevelsDataTemp[xVarsVec_i], TTranDataTemp], axis=1)
            kLevelsData.columns   = [(VarName + '_j') for VarName in kLevelsData.columns]
    
            xTemp_FWD             = pd.concat([iLevelsData, jLevelsData, kLevelsData], axis=1)
    
   
            if (len(xTemp_FWD[NN_KExcit.xTrainingVar]) > 0):
            
                if(ExcitType == 'KInel'):   
                    KInel_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact         
                    csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, 0, Molecules, Atoms, iIdx, jIdxVec, KInel_NN_FWD)
                
                elif(ExcitType == 'KExch'):      
                    KExch_NN_FWD      = np.exp( NN_KExcit.Model.predict(xTemp_FWD[NN_KExcit.xTrainingVar]) ) / InputData.MultFact
                    csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, 0, Molecules, Atoms, iIdx, jIdxVec, KExch_NN_FWD)
                           
    if(ExcitType == 'KInel'): 
        csvkinetics_KInel = write_predictiondata(ExcitType, KineticFile_KInel, csvkinetics_KInel, -2, None, None, None, None, None)
    elif(ExcitType == 'KExch'):
        csvkinetics_KExch = write_predictiondata(ExcitType, KineticFile_KExch, csvkinetics_KExch, -2, None, None, None, None, None)
    

[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes:   0%|          | 0/9399 [00:00<?, ?it/s]

[SurQCT]:    TTran =  10000.0
[SurQCT]:   Writing Kinetics in File: /home/venturi/WORKSPACE//Air_Database/Run_0D_surQCT/database/kinetics/nondim_N3_NASA_Active_Run15//T10000K/Exch_Type1_QCTRate_v0v1_all.dat


[SurQCT]:     Generating Inelastic and Exchange Rate Matrixes: 100%|█████████▉| 9397/9399 [57:23<00:00,  2.73it/s]  


KeyError: 9399